## Project's Folder Structure

```
SPEEDY-Interpoler/
├── .ipynb_checkpoints/
│   └── Untitled-checkpoint.ipynb
├── Data/
│   ├── NOAA/
│   │   ├── Atmospherical_Conditions/
│   │   │   └── NOAA NETCDF Files of Atmospherical Conditions
│   │   └── Pressure_Conditions/
│   │       └── NOAA NETCDF Files relative of Pressure on Surface information
│   └── SPEEDY/
│       └── SPEEDY model .grd Files
└──Notebooks/
    └── Required Jupyter Notebooks to perform interpolation and error estimation
```

## Imports and Path Settings

In [1]:
import os
import struct
from pathlib import Path

import numpy as np
from scipy.interpolate import griddata
from netCDF4 import Dataset

In [87]:
path = Path.cwd()/'../Data/NOAA/Atmospherical_Conditions'
pressure_path = Path.cwd()/'../Data/NOAA/Pressure_Conditions'
files = os.listdir(path)
list.sort(files)

## Model Settings

In [3]:
# NOAA Default pressure leves
pressure_levels = [1,2,3,5,7,9,11]
pressure_values = [925,850,700,500,300,200,100]

# Grids definition for SPEEDY and NOAA
X_speedy_long = np.linspace(360-3.75,0,96)
Y_speedy_lat = np.array("-87.159 -83.479 -79.777 -76.070 -72.362 -68.652 -64.942 -61.232 -57.521 -53.810 -50.099 -46.389 -42.678 -38.967 -35.256 -31.545 -27.833 -24.122 -20.411 -16.700 -12.989 -9.278 -5.567 -1.856 1.856 5.567 9.278 12.989 16.700 20.411 24.122 27.833 31.545 35.256 38.967 42.678 46.389 50.099 53.810 57.521 61.232 64.942 68.652 72.362 76.070 79.777 83.479 87.159".split(" "))
Y_speedy_lat = Y_speedy_lat.astype(np.float32)

X_noaa_long = np.linspace(360-2.5,0,144)
Y_noaa_lat = np.linspace(-90,90,73)
X_grid_noaa, Y_grid_noaa = np.meshgrid(X_noaa_long,Y_noaa_lat)

## Functions

To find more information about griddata go [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.griddata.html)

In [89]:
def ReadData(path, variable_name,hour_instant):
    '''
        input variables
            path          : path of the variable to be read
            variable_name : name of the variable
            hour_instant  : refers to NOAA instant of time which data will be readed.
        output variable
            variable_array: 3-dimensional array with all the pressure levels of the given variable
    '''
    variable = Dataset(path,'r')
    variable_array = np.array(variable[variable_name][hour_instant,pressure_levels,:,:])
    return variable_array

def Interpoler(variable_grid, method = 'nearest'):
    '''
        input variables:
            variable_grid : variable in specific level to be spatially interpolated
            method        : method to use for the interpolation function (linear, nearest, cubic)
        output variable
            interp_noaa_speedy: speedy grid for certain level
    '''
    interp_noaa_speedy = griddata((X_grid_noaa.ravel(),Y_grid_noaa.ravel()), variable_grid.ravel()
                              , (X_speedy_long[None,:], Y_speedy_lat[:,None]), method=method)
    return interp_noaa_speedy

def relative2specific(T,RH,p):
    '''
    Parameters
    ----------
        T : Temperature in K.
        RH : Relative humidity in percentage [0,100].
        p : Preassure in mbar.

    Returns
    -------
        specific humidity (dimensionless)

    '''  
    T-=273.15
    p*=100
    RH/=100
    e_s=611.21*np.exp((18.687-T/234.5)*(T/(T+257.14)))
    e=e_s*RH
    w=287.058/461.5*e/(p-e)
    return w/(w+1)

## Interpolation

**Variables order in list**  
0 : Air Temperature (7 lvls)  
1 : RHumidity (7 lvls)  
2 : Uwnd (7 lvls)  
3 : Vwnd (7 lvls)  
4 : Pressure (1 lvls)  
5 : Specific Humidity (7 lvls)

### Atmospherical Conditions interpolation

In [90]:
variables_list = list()
for file in files:
    variable_name = file.split(".")[0]
    variable_array = ReadData(path/file, variable_name,0)
    variable_levels = dict()
    for i in range(len(pressure_levels)):
        variable_level = variable_array[i,:,:]
        variable_levels[variable_name+"_"+str(pressure_levels[i])] = Interpoler(variable_level)
    variables_list.append(variable_levels)

### Pressure interpolation

In [35]:
variable_levels = dict()
variable_pressure = Dataset(pressure_path/'pres.sfc.2020.nc','r')
variable_name = 'pres'
variable_array_pressure = np.array(variable_pressure[variable_name][3,:,:])
variable_levels[variable_name+"_1"] = Interpoler(variable_array_pressure)
variables_list.append(variable_levels)

### Relative Humidity to Specific Humidity Convertion

In [42]:
variable_levels = dict()
for j,i in enumerate(pressure_levels):
    RH_data = variables_list[1]['rhum_'+str(i)].copy()
    air_T_data = variables_list[0]['air_'+str(i)].copy()
    pressure = pressure_values[j]
    #print(RH_data.shape, air_T_data.shape, pressure)
    variable_levels['shum_'+str(i)] = np.vectorize(relative2specific)(air_T_data,
                                                                      RH_data,
                                                                      pressure)
variables_list.append(variable_levels)

## Convert to GRD vector form

In [43]:
component_names = list()
for i in range(len(variables_list)):
    print(list(variables_list[i].keys()))
    component_names.append(list(variables_list[i].keys()))

['air_1', 'air_2', 'air_3', 'air_5', 'air_7', 'air_9', 'air_11']
['rhum_1', 'rhum_2', 'rhum_3', 'rhum_5', 'rhum_7', 'rhum_9', 'rhum_11']
['uwnd_1', 'uwnd_2', 'uwnd_3', 'uwnd_5', 'uwnd_7', 'uwnd_9', 'uwnd_11']
['vwnd_1', 'vwnd_2', 'vwnd_3', 'vwnd_5', 'vwnd_7', 'vwnd_9', 'vwnd_11']
['pres_1']
['shum_1', 'shum_2', 'shum_3', 'shum_5', 'shum_7', 'shum_9', 'shum_11']


### Sanity checks

The order must be: uwnd, vwnd, air, shum, pres

In [62]:
max_list = list()
min_list = list()
for i in component_names[0]:
    max_list.append(variables_list[0][i].max())
    min_list.append(variables_list[0][i].min())
print('air')
print(max_list)
print(max(max_list))
print(min_list)
print(min(min_list))

air
[310.5, 304.71, 288.5, 273.0, 247.0, 236.5, 234.20999]
310.5
[231.8, 231.2, 227.5, 222.3, 209.8, 199.70999, 188.6]
188.6


In [61]:
max_list = list()
min_list = list()
for i in component_names[2]:
    max_list.append(variables_list[2][i].max())
    min_list.append(variables_list[2][i].min())
print('uwnd')
print(max_list)
print(max(max_list))
print(min_list)
print(min(min_list))

uwnd
[25.199997, 27.809998, 34.699997, 54.899994, 95.10999, 85.60999, 54.809998]
95.10999
[-20.800003, -18.399994, -18.699997, -20.399994, -23.899994, -18.399994, -20.600006]
-23.899994


In [49]:
max_list = list()
min_list = list()
for i in component_names[3]:
    max_list.append(variables_list[3][i].max())
    min_list.append(variables_list[3][i].min())
print('vwnd')
print(max_list)
print(max(max_list))
print(min_list)
print(min(min_list))

vwnd
[29.399994, 28.0, 28.399994, 35.0, 55.309998, 49.61, 29.399994]
55.309998
[-22.600006, -23.199997, -25.300003, -31.800003, -62.0, -48.0, -26.399994]
-62.0


In [51]:
max_list = list()
min_list = list()
for i in component_names[5]:
    max_list.append(variables_list[5][i].max())
    min_list.append(variables_list[5][i].min())
print('shum')
print(max_list)
print(max(max_list))
print(min_list)
print(min(min_list))

shum
[0.02414183614302363, 0.01893843207610807, 0.011877547628140524, 0.005946422493147153, 0.0009729118541460497, 0.00018875902855863325, 2.0238476133343783e-05]
0.02414183614302363
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.0


In [64]:
print('press')
print(variables_list[4]['pres_1'].max())
print(variables_list[4]['pres_1'].min())

press
104900.0
50230.0


### Convert to list in order to write it in GRD file

# REMEMBER TO FLIPFLIP DATA!!!!!

In [79]:
result_list = list()
#uwnd
for level in component_names[2]: 
    result_list.extend(np.reshape(variables_list[2][level], 96*48).tolist())
#vwnd
for level in component_names[3]: 
    result_list.extend(np.reshape(variables_list[3][level], 96*48).tolist())
#air
for level in component_names[0]:
    result_list.extend(np.reshape(variables_list[0][level], 96*48).tolist())
#shum
for level in component_names[5]:
    result_list.extend(np.reshape(variables_list[5][level], 96*48).tolist())
#pres
result_list.extend(np.reshape(variables_list[4]['pres_1'], 96*48).tolist())

In [80]:
len(result_list)

133632

### File Writing

In [82]:
fout = open('test3.grd', 'wb')
for i in result_list:
    fout.write(struct.pack('>f',i))
fout.close()